# Update for Preseason 11 items

Gathering a lot of matches from 10.24 patch, and analyzinf the frequency of items per role.

In [1]:
import pymongo

client = pymongo.MongoClient()
db = client.game_ps_11

mongoTable = db["gameEuw"]

In [3]:
#Initializing the target composition
role_composition = set(["JUNGLE_NONE","TOP_SOLO","MIDDLE_SOLO","BOTTOM_DUO_CARRY","BOTTOM_DUO_SUPPORT"])

import requests
#Initializing the itemsPlayed array
itemsPlayed = {}

r = requests.get('https://ddragon.bangingheads.net/cdn/10.24.1/data/en_US/item.json')
dataItems = r.json()
for r in role_composition:
    itemsPlayed[r] = {}
    for i in dataItems['data']:
        itemsPlayed[r][int(i)] = 0


for g in mongoTable.find({ "gameDuration": { "$gt": 900 }, "mapId":11 }):
    
    
    #Get teams that have a perfect metagame composition
    positionsByTeam = {100:[],200:[]}
    for p in g['participants']:
        positionsByTeam[p['teamId']].append(p['timeline']['lane']+"_"+p['timeline']['role'])
    teamOK = {}
    teamOK[100] = role_composition == set(positionsByTeam[100])
    teamOK[200] = role_composition == set(positionsByTeam[200])
    
    #Get items used
    for p in g['participants']:
        if teamOK[p['teamId']]:
            
            #For all item slots
            for i in range(0,7):
                #If there is an item in this slot
                if p['stats']['item'+str(i)] > 0:
                    #Increment item count for the specific role
                    itemsPlayed[p['timeline']['lane']+"_"+p['timeline']['role']][p['stats']['item'+str(i)]] += 1

In [4]:
import pandas as pd
dfItemsFrequency = pd.DataFrame(itemsPlayed).T
dfItemsFrequency

,1001,1004,1006,1011,1018,1026,1027,1028,1029,1031,...,7013,7014,7015,7016,7017,7018,7019,7020,7021,7022
BOTTOM_DUO_SUPPORT,17425,14812,4077,24918,2260,39887,3141,33570,23900,5830,...,634,82,4,1,42,8,403,103,304,542
MIDDLE_SOLO,13307,333,98,13267,14411,40240,3062,15481,8174,7189,...,1453,72,22,8,31,91,2,0,0,44
TOP_SOLO,12170,176,854,27343,10566,16895,1073,43523,27845,25278,...,11,1,16,5,22,24,10,1,0,0
JUNGLE_NONE,13263,282,1044,24316,11269,12015,496,45584,29149,28732,...,39,2,491,51,173,474,3,0,5,8
BOTTOM_DUO_CARRY,10597,67,32,1172,65301,3524,828,3219,2750,5817,...,70,8,0,2,313,225,0,0,0,4


In [5]:
overUsedItems = {
    "BOTTOM_DUO_SUPPORT":[],
    "JUNGLE_NONE":[],
    "TOP_SOLO":[],
    "BOTTOM_DUO_CARRY":[],
    "MIDDLE_SOLO":[]
}

mostlyUsedItems = {
    "BOTTOM_DUO_SUPPORT":[],
    "JUNGLE_NONE":[],
    "TOP_SOLO":[],
    "BOTTOM_DUO_CARRY":[],
    "MIDDLE_SOLO":[]
}

underUsedItems = {
    "BOTTOM_DUO_SUPPORT":[],
    "JUNGLE_NONE":[],
    "TOP_SOLO":[],
    "BOTTOM_DUO_CARRY":[],
    "MIDDLE_SOLO":[]
}

for itemId in dfItemsFrequency:
    
    for role,j in enumerate(dfItemsFrequency[itemId]):
        
        if j>(dfItemsFrequency[itemId].sum() * 0.9):
            overUsedItems[dfItemsFrequency[itemId].index[role]].append(str(itemId))
            
        elif j>(dfItemsFrequency[itemId].sum() * 0.6):
            mostlyUsedItems[dfItemsFrequency[itemId].index[role]].append(str(itemId))
            
        elif j<(dfItemsFrequency[itemId].sum() * 0.1):
            underUsedItems[dfItemsFrequency[itemId].index[role]].append(str(itemId))

Comparing to old data, to see if there are some incompatiblities between before and after.

In [10]:
import json

old_data = json.load(open("../roleml/data/constants.json","r"))

In [14]:
for i in old_data["most_used_items"]:
    for item in underUsedItems[i]:
        if item in old_data["most_used_items"][i]:
            print(i, item)

In [17]:
for i in old_data["decently_used_items"]:
    for item in underUsedItems[i]:
        if item in old_data["decently_used_items"][i]:
            print(i, item)

In [16]:
for i in old_data["least_used_items"]:
    for item in overUsedItems[i]:
        if item in old_data["least_used_items"][i]:
            print(i, item)

In [27]:
for i in old_data["least_used_items"]:
    for item in mostlyUsedItems[i]:
        if item in old_data["least_used_items"][i]:
            print(i, item)

No problem found, simply adding the new data to the old will work and allow not to care about the version.

In [23]:
new_most_used_items = {
    i:sorted(list(set(old_data["most_used_items"][i] + overUsedItems[i]))) for i in old_data["most_used_items"]
}

In [26]:
new_decently_used_items = {
    i:sorted(list(set(old_data["decently_used_items"][i] + mostlyUsedItems[i]))) for i in old_data["decently_used_items"]
}

new_least_used_items = {
    i:sorted(list(set(old_data["least_used_items"][i] + underUsedItems[i]))) for i in old_data["least_used_items"]
}

In [28]:
new_data = old_data
new_data["most_used_items"] = new_most_used_items
new_data["decently_used_items"] = new_decently_used_items
new_data["least_used_items"] = new_least_used_items

In [30]:
json.dump(new_data, open("../roleml/data/constants.json","w"))